# CAO Points 2021 analysis

## Purpose
##### The purpose of this notebook is to analyse the CAO points required for Level 8 courses in 3rd level institutions in Ireland. 

### Library imports
##### The libraries used in the analysis are numpy, pyplot and Element Tree

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import xml.etree.ElementTree as ET